In [7]:
import pandas as pd
import numpy as np 
from datetime import timedelta

dfDatabase = pd.read_csv("//wsl.localhost/Arch/home/catz/KromosData/Database.csv")
dfSprint = pd.read_csv("//wsl.localhost/Arch/home/catz/KromosData/2023.csv")

In [8]:
# Sprint
# Rename columns
temp_dfSprint = dfSprint.rename(columns={'Task ID': 'Subtask ID', 'Task Name': 'Subtask name', 'Parent ID': 'Task ID', 'Assignee': 'Worker', 'tags': 'Activity', 'Data de entrada (date)': 'DG Check-in date'})

# Drop trash columns 
columns_to_drop = ['Parent Name', 'Parent URL', 'Status', 'Task Content', 'Latest Comment', 'Comment Count', 'Assigned Comment Count', 'Due Date', 'Due Date Text', 'Start Date', 'Start Date Text', 'Date Created', 'Date Created Text', 'Date Updated', 'Date Updated Text', 'Date Closed Text', 'Date Done', 'Created By', 'Space', 'Folder', 'List', "Subtask ID's", "Subtask URL's", 'Lists', 'Sprints', 'Linked Tasks', 'Linked Docs', 'Time Estimate (hours)', 'Time In Status', 'Points Estimate', 'Points Estimate Rolled Up', 'Tempo no DG (formula)']
temp_dfSprint = temp_dfSprint.drop(columns_to_drop, axis=1)

# Miss error DG
temp_dfSprint.loc[temp_dfSprint['Subtask ID'] == '86a1mqcnr', 'Date Closed'] = '2023-12-14T09:45:00.000-03:00'

# Data cleaning
temp_dfSprint['Worker'] = temp_dfSprint['Worker'].replace('\\[', '', regex=True).replace('\\]', '', regex=True).replace('', np.nan)
temp_dfSprint['Priority'] = temp_dfSprint['Priority'].replace('none', np.nan)
temp_dfSprint['Activity'] = temp_dfSprint['Activity'].replace('\\[', '', regex=True).replace('\\]', '', regex=True).replace('', np.nan)

# Database
# Rename columns
temp_dfDatabase = dfDatabase.rename(columns={'Assignee': 'Seller', 'Cores orçadas (short text)': 'Budgeted Colors', 'Contato do cliente (text)': 'Customer Contact', 'Máquinas (labels)': 'Machines', 'Planta (tasks)': 'Blueprint', 'Faca (tasks)': 'Knife Link', 'Especificação técnica (url)': 'Technical specification', 'Ultima OS (number)': 'Last OS', 'Orçamento (short text)': 'Budget', 'Frasco (tasks)': 'Bottle Link', 'Cor (tasks)': 'Color Link', 'Arte (url)': 'Artwork Link', 'Substrato (drop down)': 'Substrate', 'Cliente (short text)': 'Client name'})

# Drop trash columns 
columns_to_drop = ['Task Content', 'Priority', 'Latest Comment', 'Comment Count', 'Assigned Comment Count', 'Due Date', 'Due Date Text', 'Start Date','Start Date Text', 'Date Created', 'Date Created Text', 'Date Updated', 'Date Updated Text', 'Date Closed', 'Date Closed Text', 'Date Done', 'Created By', 'Space', 'Folder', 'List', "Subtask ID's", "Subtask URL's", 'tags', 'Lists', 'Sprints', 'Linked Tasks', 'Linked Docs', 'Time Logged', 'Time Estimate (hours)', 'Time In Status', 'Points Estimate', 'Points Estimate Rolled Up', 'Duração (drop down)', 'Data de entrada (date)', 'ID-Vendedor (number)']
temp_dfDatabase = temp_dfDatabase.drop(columns_to_drop, axis=1)

# Data cleaning
temp_dfDatabase['Knife Link'] = temp_dfDatabase['Knife Link'].replace('\\[', '', regex=True).replace('\\]', '', regex=True).replace('', np.nan)
temp_dfDatabase['Machines'] = temp_dfDatabase['Machines'].replace('\\[', '', regex=True).replace('\\]', '', regex=True).replace('', np.nan)
temp_dfDatabase['Seller'] = temp_dfDatabase['Seller'].replace('\\[', '', regex=True).replace('\\]', '', regex=True).replace('', np.nan)
temp_dfDatabase['Color Link'] = temp_dfDatabase['Color Link'].replace('\\[', '', regex=True).replace('\\]', '', regex=True).replace('', np.nan)
temp_dfDatabase['Bottle Link'] = temp_dfDatabase['Bottle Link'].replace('\\[', '', regex=True).replace('\\]', '', regex=True).replace('', np.nan)

# Code create 
temp_dfDatabase['Task Code'] = temp_dfDatabase['Task Name'].apply(lambda x: '-'.join(x.split('-', 2)[:2]))

In [9]:
# Converter to date time
temp_dfSprint['DG Check-in date'] = pd.to_datetime(temp_dfSprint['DG Check-in date'])
temp_dfSprint['Date Closed'] = pd.to_datetime(temp_dfSprint['Date Closed'])

# TimeDelta to second
temp_dfSprint['Time Logged'] = temp_dfSprint['Time Logged'].astype('datetime64[ns]')
temp_dfSprint['Time Logged'] = (temp_dfSprint['Time Logged'].dt.hour * 60 + temp_dfSprint['Time Logged'].dt.minute) * 60 + temp_dfSprint['Time Logged'].dt.second

# New column of working days
temp_dfSprint['Working days'] = np.busday_count(
    begindates=temp_dfSprint['DG Check-in date'].values.astype('datetime64[D]'),
    enddates=temp_dfSprint['Date Closed'].values.astype('datetime64[D]')
)

# The merge between dataframes
merged_df = pd.merge(temp_dfSprint, temp_dfDatabase, on='Task ID', how='inner')

merged_df.to_csv(r"C:\Users\caiop\Downloads\Sprint_Refactored.csv", index=False)
# merged_df.head(200)

#temp_dfSprint.to_csv(r"//wsl.localhost/Arch/home/catz/KromosData/Sprint_Teste.csv", index=False)
#temp_dfDatabase.to_csv(r"//wsl.localhost/Arch/home/catz/KromosData/Database_Teste.csv", index=False)

C:\Users\caiop\AppData\Local\Temp\ipykernel_51232\44291030.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  temp_dfSprint['Time Logged'] = temp_dfSprint['Time Logged'].astype('datetime64[ns]')
